## Посмотреть содержимое датасета задачи

In [3]:
import os

DATASET_ROOT = "../dataset"
TASK_DATASET = os.path.join(DATASET_ROOT, "ok-group-recommend")

os.listdir(TASK_DATASET)

['sample_submission.csv', 'test_customer_ids.csv', 'train_df.tsv']

## Прочитать содержимое тестового набора и вывести первые 5 строк

In [37]:
import pandas as pd

test = pd.read_csv(os.path.join(TASK_DATASET, "test_customer_ids.csv"))
test.head(5)

,customer_id
0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...
1,73821118fc33500efaa6b1adf8ab0e9d314abb15f62603...
2,6381971c002097a94b8d7a03d9dc3e9ff7872a52c4764a...
3,250d49b476af0c7d2b23dd39cbb6edff39d44c64f8ebc0...
4,2da339a6bfe7791329ca8eb0a11544d4a9fb4c89572716...


## Прочитать файл примера решения и вывести первые 5 строк

## Реализовать код, формирующий файл решения
В демо-варианте делается простое копирование примера

In [5]:
df = pd.read_csv(os.path.join(TASK_DATASET, "train_df.tsv"), sep='\t')
df.head(5)

,Unnamed: 0,community_id,description,customers_count,messages_count,type,region_id,themeid,business_category,business_parent,customer_id,status,join_request_date
0,0,00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc75...,"Ванга сказала: ""Выживет пчела - выживет челове...",2966,1,7,NaN,NaN,NaN,NaN,947224211267aefcc2e3e9c524fdf46ce329bc638e8bf1...,A,NaN
1,1,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,b935c3390d82df612de19591d2dd16e1eff8e660746bb3...,A,NaN
2,2,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,ff873dee9234ee290d92eb6c15d7179aabf0653dbed3b3...,A,NaN
3,3,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,737ef0e849feda6cdfd350d19fcedaf17aebb757c0a1fc...,A,NaN
4,4,097e4dae68029ecc3c6183ed4386aff71dcac227cbaff7...,Читать внимательно!!! Причины попадания в чёрн...,1035,1,7,1.041779e+10,NaN,FAN_CLUB,BLOG,780fb8c178af972826ac0f84e42ac4c6c1dcdeeec0c4dd...,A,NaN


In [6]:
df = df.drop(['description', 'customers_count', 'messages_count', 'type', 'region_id', 'themeid', 'business_category',
             'business_parent', 'join_request_date'], axis=1)

In [7]:
df = df.drop(['Unnamed: 0'], axis=1)

In [8]:
df = df.drop(['status'], axis=1)

In [9]:
comm = pd.DataFrame(df['community_id'].unique()).rename(columns={0: 'community_id'})
comm['community_id_new'] = comm.index
cust = pd.DataFrame(df['customer_id'].unique()).rename(columns={0: 'customer_id'})
cust['customer_id_new'] = cust.index

In [10]:
df = pd.merge(df, comm)
df = pd.merge(df, cust)

In [11]:
cust_l = list(cust['customer_id'])

In [36]:
comm_l = list(comm['community_id'])

In [12]:
from implicit.nearest_neighbours import (
    BM25Recommender,
    CosineRecommender,
    TFIDFRecommender,
    bm25_weight,
)
from scipy.sparse import csr_matrix
from lightfm.cross_validation import random_train_test_split
from sklearn.model_selection import train_test_split
import numpy as np

In [13]:
data_full = csr_matrix((np.ones((df.shape[0],)).astype(float),
                        (df['customer_id_new'].values,
                        df['community_id_new'].values)))

In [14]:
data_train, data_test = random_train_test_split(data_full)

In [16]:
cr = CosineRecommender()

cr.fit(data_train)

/opt/conda/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.7726864814758301 seconds
  warnings.warn(


  0%|          | 0/149114 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [18]:
from tqdm import tqdm

In [19]:
test

,customer_id
0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...
1,73821118fc33500efaa6b1adf8ab0e9d314abb15f62603...
2,6381971c002097a94b8d7a03d9dc3e9ff7872a52c4764a...
3,250d49b476af0c7d2b23dd39cbb6edff39d44c64f8ebc0...
4,2da339a6bfe7791329ca8eb0a11544d4a9fb4c89572716...
...,...
69041,606d6caf18a6209acaa46a1a0233a4bf22bc9885d70397...
69042,a71392c55f8bace9bd108a1acbf9c2335d700fb041966b...
69043,7b6d03389208df699aa116563dc5c45cbf5c725b47b415...
69044,8d1c564ba9f96dafb0a856fbc65fc4f9fc020863fe3b3f...


In [49]:
sub = pd.DataFrame(columns=['customer_id','community_id_1','community_id_2','community_id_3','community_id_4','community_id_5','community_id_6','community_id_7'])

In [50]:
sub

,customer_id,community_id_1,community_id_2,community_id_3,community_id_4,community_id_5,community_id_6,community_id_7


In [51]:
for i, row in tqdm(test.iterrows()):
    
    id = cust_l.index(row['customer_id'])
    #test['customer_id'][i] = id
    rect = cr.recommend(id, data_full[id].astype(float))[0][:7]
    rect = [row['customer_id']]+[comm_l[i] for i in rect]
    sub.loc[len(sub)] = rect
    
    
        


69046it [05:46, 199.41it/s]


In [52]:
sub

,customer_id,community_id_1,community_id_2,community_id_3,community_id_4,community_id_5,community_id_6,community_id_7
0,d811a09d435ac2d1b1ed46e272405af10933b4711f4564...,e2d8710c3ca6ccb07ca3fbf66f0530be94903a58f5c2a8...,39fb57eaf8ebcb8008e04b197aafee1b319a98be7c585b...,881ca660e8ae340c56dba18e4502c7c149846f2d11708d...,d6ee22c7c0561950b19f4d76342e00993eb90cc689c6da...,4efedd3dc8834de75c0def70c38eee9e71a2ab899c0869...,b22e62fafd0d39725937879df9fcb9412432b9ff5cf009...,afcc530458d5118edd3fba7b2a885655626e9ee9cf1634...
1,73821118fc33500efaa6b1adf8ab0e9d314abb15f62603...,fc781628c86a7ee722f0efc9b75c9bf5a1360855a1e665...,d1af495ca20772cd9fcb43e8c27da695b5e9d93c64fc39...,f37f3f1b66e18fc5a425e338baced679c264de8e995e36...,53747ccd59fc17eac767624975cdd2de14aa6efa6090ae...,0380de4e396e8d4bd903d58299f67d06a84d9e8350250a...,c1ecc505d7c96ebbbc12129592f02928806dce4ade2023...,a437b788fc91efab69b24b19e18d88142667d0fc39c48a...
2,6381971c002097a94b8d7a03d9dc3e9ff7872a52c4764a...,9ecd44db17abc1fc665d38db272893274d5761665eb27c...,d42e10f2610272c545292b60844df2eb7a3ba47d6b4829...,ff09872273583edd92b50c43ad4ba41ebb9fe0b1210791...,ce7a456428f786c44c3e27c039c4299885da5430ccfd05...,66d19c807bb7ea8a13c9fb8aab294b2380f5d4ecf8e953...,9b73d8f7440da961df0bc04a4ac228ee3f01123b5d19cc...,739bbe0a10ad6d28d092974c04d92626ae21e68b69dc24...
3,250d49b476af0c7d2b23dd39cbb6edff39d44c64f8ebc0...,2165f7bfb5b82ede0cf8f3be218c7f57ef2cd1e282ae9c...,a8b15b543469eebe27c67c0c5897f6dc18e160cc5997c0...,892f2c17a6b9684c4dfd9834c0c6c8e7694c9faa00a0b3...,16183b4e07446d00c10d26deb9a1d62814d67fb6039b59...,6729664ac03bd48062612647597252d30d097b79e8308b...,a8d0b95bb7330d3a1171c439c5db88e1a3671edfbca950...,7ed0a5203360da0630e5ad51f2037a4d81b97e000d1cf1...
4,2da339a6bfe7791329ca8eb0a11544d4a9fb4c89572716...,3823a5e13dfd871d3ff72d5a69af9b77466d604f43cecd...,4d7cf04c8e56f0797012064220d0a45509a1264bc17f58...,1717cd8dbd989b4bd56009c53db75d564b252a3f3ed722...,14d2fb073c9a2929f1fd9919cbbae50a5c597d2459d7f5...,b8ed0e918d9f11140c70ef08be19c197917f4f4455f255...,b927c9e7135a558e2ed237459ccf02230e6c2c3400f776...,2297e8137751faa879da9ceea4a47423a9eda35d9b0278...
...,...,...,...,...,...,...,...,...
69041,606d6caf18a6209acaa46a1a0233a4bf22bc9885d70397...,274445fd08bfa77e358b0599c1adb91e6900d58335f0bb...,face1187302b0000986e09146fe04bdb5f39c4cd0ab33a...,57d99404c90c9344d3aec2c15ffb7b1a0f46c8a4f7157a...,a372c294e86415520031005bba83bb11c94eab26b3727c...,00fb053b79e838767745c8aac82e1bb90efead133447ae...,bfbe0ab1789db400d4090745c5c54a6f09102048f04aa8...,8792b896fc5d399bbacc274bc5deac4f25f6caadbf28cb...
69042,a71392c55f8bace9bd108a1acbf9c2335d700fb041966b...,6570dd9605aa01ff78a0224a47cddc379bbf7b1db10d28...,8c3f63d3b0e551bbe6434b89cbc53045026a2a9e56276f...,e88c1aa3c9ca59917ddf7ecd022fa7da468fd3f342ae77...,8c5ff31e392d00ef67074b7a086ddd1df5a17cd2f27112...,ff5b7a1bc51537685973e6c31d8f92c55ac474a1da492e...,0aebd3b9934b9ec88cb78a99256bab2987798b93ff5a8d...,75f03af4f51c4c9fdf73fb9dd495964b776f2adc5b0154...
69043,7b6d03389208df699aa116563dc5c45cbf5c725b47b415...,5280bd59e190e9897b8481f1751f5508b76188da909b83...,f6a7fbac98fa02e64db8cb671ae5b529fa944d1f41a7f6...,ab5b19507c5a96c274a875f90d372882bde7956796df51...,e015fa5e6ef36af3ea62a91263c0fba81bd56794a9716a...,7723809a0370191d10dce06820ff9f132648491fd3b092...,de2b374004d89c21ab753e06dc73c2740403709edae8b2...,d7a49cba22dd3fba7de0ece2c8938d05ae38e03a08cb9f...
69044,8d1c564ba9f96dafb0a856fbc65fc4f9fc020863fe3b3f...,311db6eae03a8ae679ec1de8c2321a120938b3eaabec93...,3bfe238a1001285a82d9c8ca05d346322ff88d9a46e180...,84b501b676c52df6a4875c896d5b4dcc6130f5a69c8f73...,51b354f4db4e89508fad99ff9b466db96a307c9d1a1941...,7e00d5b79bd18c67a35bc4622a06c24d9b2a01e815ece5...,4cd507db4bc028745427889e579bbf3591662d2885a088...,6c34c5baa7127cb397315a4bd013db70872be8eee6dd2f...


In [21]:
test

,customer_id
0,18560
1,27401
2,14225
3,340
4,9820
...,...
69041,56961
69042,46391
69043,63840
69044,30935


In [31]:
rect = cr.recommend(18560, data_full[18560].astype(float))

In [32]:
len(rect)

2

In [33]:
for i in rect:
    print(i)
    

[136663 103147 141575 139231 135217 134537 129758 117518  77786  13108]
[0.73029674 0.49507377 0.48507125 0.48507125 0.48507125 0.48507125
 0.48507125 0.48507125 0.48507125 0.43450579]


## Сохранить сформированное решение в файл

In [53]:
sub.to_csv("my_submit_demo_task.csv", index=False)

## Заархивировать решение (если в условии задачи указано такое требование)

In [54]:
from zipfile import ZipFile, ZIP_DEFLATED
 
with ZipFile("my_submit_demo_task.zip", "w", compression=ZIP_DEFLATED, compresslevel=5) as submit_zip:
    submit_zip.write("my_submit_demo_task.csv")
    print ("done")

done


## Остается скачать файл my_submit_demo_task.zip из каталога work и загрузить его в проверяющую систему All Cups

Ссылка на задачу в All Cups: https://cups.online/ru/workareas/vserosii_2023/789/1539?is_sandbox=1